**This notebook contains a script that sends learnership contract templates to sponsors for signing**

**Add Local Library to path**

In [252]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    sys.path.append(module_path + '/local_library')

**Import Libraries**

In [253]:
import pandas as pd
import requests
import json
from datetime import date
from local_library import import_worksheet
from local_library import export_worksheet
from local_library import send_bulk_signrequest

**Create Local Library**

In [254]:
#split a column into multiple columns
def split_column_num(target_df, column, column_len):
    
    #create list from 0 to column_len
    n_list = list(range(0, column_len))
    
    #create and populate columns
    for i in n_list:
        target_df[f'{column}_{i+1}'] = [elem[i] if i < len(elem) else '' for elem in target_df[column]]
        
    return


#split column into two binary columns
def column_to_binary(target_df, column, specify='no'):
    
    #convert column to string
    target_df[column] = target_df[column].astype(str)
    
    #strip whitespace
    target_df[column] = target_df[column].str.strip()
    
    #create and populate binary columns
    target_df[f'{column}_yes'] = ['X' if elem.lower() == 'yes' else '' for elem in target_df[column]]
    target_df[f'{column}_no'] = ['X' if elem.lower() == 'no' else '' for elem in target_df[column]]
    
    return


#split address into component parts
def split_address(target_df, column):
    
    #set column to type string
    target_df[column] = target_df[column].astype(str)
    
    #split address into street, area, city and strip
    for i in range(1,4):
        if len(list(target_df[column].str.split(","))[0]) > i - 1:
            target_df[f"{column}_{i}"] = target_df[column].str.split(",").str[i-1] + ","
            target_df[f"{column}_{i}"] = target_df[f"{column}_{i}"].str.strip()
        else:
            target_df[f"{column}_{i}"] = " "
    
    #split addrss into area code and strip
    if len(list(target_df[column].str.split(","))[0]) > 3:
        target_df[f"{column}_post_code"] = target_df[column].str.split(",").str[3:].str.join("")
        target_df[f"{column}_post_code"] = target_df[f"{column}_post_code"].str.strip()
    else:
        target_df[f"{column}_post_code"] = " "
    
    return

#convert phone numbers
def convert_phone(target_df, column):
    target_df[column] = target_df[column].astype(str)
    target_df[column] = target_df[column].astype(float)
    target_df[column] = target_df[column].astype(int)
    target_df[column] = target_df[column].astype(str)
    
    return


#convert column type to string
def convert_columns_type_string(target_df, columns):
    for column in columns:
        target_df[column] = target_df[column].astype(str)

    return


#split columns based on column values
def split_column(target_df, column):
    new_columns = target_df[column].unique()
    for new_column in new_columns:
        target_df[new_column] = ['X' if elem == new_column else '' for elem in target_df[column]]
        
    return


#split column based on status
def split_column_status(target_df, column, value):
    #mark if citizen yes
    target_df[f"{column}_yes"] = ['X' if elem == value else '' for elem in target_df[column]]

    #mark if citizen no
    target_df[f"{column}_no"] = ['X' if elem != value else '' for elem in target_df[column]]

    #specify citizenshp status
    target_df[f"{column}_specify"] = [elem if elem != value else '' for elem in target_df[column]]
    
    return


#createe prefill tags
def create_tags(row, columns):
    prefill_tags = []
    for column in columns:
        tags = {}
        tags["external_id"] = column
        tags["text"] = row[column]
        prefill_tags.append(tags)
        
    return prefill_tags


#add static column
def add_static_columns(target_df, column_dict):
    for key in column_dict.keys():
        target_df[key] = column_dict[key]
        
    return


#subset a dataframe
def subset_dataframe(target_df, column, subset_array):
    return target_df[target_df[column].isin(subset_array)]


#combine sponsor and cohort dataframes
def combine_dataframes(target_df, sponsor_df):

    for column in sponsor_df.columns:
        target_df.loc[:,column] = sponsor_df[column].values[0]
    
    return

#replace nan values
def replace_nan(target_df, columns):
    target_df.loc[:,columns] = ["" for elem in columns]
    return

**Import Worksheets**

In [255]:
#import sponsorship data
sponsor_df = import_worksheet("Learnership Agreement Form (Responses)", "Form Responses 1")

#import tag ids
tag_ids_df = import_worksheet("Tag IDs","Sponsor")

**Update Sponsor Column Names**

In [256]:
column_names = {'Legal Name of Company:': 'employer_legal_name',
                           'Trading Name of Company (if different from Legal Name):':'employer_trading_name',
                           'Business Address:':'business_address',
                           'Postal Address (if different from Business Address):':'business_postal_address',
                           'Are you liable for skills development levy?':'skills_development',
                           'If yes, what is your SDL number?':'sdl',
                           'Name of SETA with which you are registered:':'seta_name',
                           'Are you acting as the Lead Employer? (the answer to this should be yes if you are the sponsor)': 'lead_employer',
                           'Contact Person responsible for Learnership:': 'learnership_contact',
                           'Work Phone:': 'employer_phone',
                           'Work Fax:':'employer_fax',
                           'E-mail Address:': 'employer_email',
                           'Full Name of representative responsible for signing the Learnership Agreement:':'employer_fullname',
                           'Initials of representative responsible for signing the Learnership Agreement:':'employer_initials',
                           'Would you like WTC_ to submit the Learnership Registration documents to your SETA on your behalf?':'delegate',
                           'If you answered Yes to the above question, has the WTC_ Letter of Delegation been signed?':'delegation_letter',
                            'Will you be utilising the WTC_ Learnership Employment Contract Template or providing your own contract template for Learnership registration?':'wtc_contract',
                           'Do you have a preferred contact person for Learnership registrations with your SETA?':'seta_contact',
                            'Standard Industrial Classification Code (sic code if applicable)':'sic',
                           'Company Registration Number':'company_registration_number',
                           'Company Registration Date':'company_registration_date',
                            'Contact Person Designation':'employer_designation'}

sponsor_df = sponsor_df.rename(columns=column_names)

**Fix Phone**

In [257]:
sponsor_df['employer_phone'] = sponsor_df['employer_phone'].astype(str)
sponsor_df['employer_phone'] = sponsor_df['employer_phone'].str.split(".")[0][0]

**Fix SIC Code**

In [258]:
sponsor_df['sic'] = sponsor_df['sic'].astype(str)
sponsor_df['sic'] = sponsor_df['sic'].str.split(".")[0][0]

**Get Specific Sponsor**

In [259]:
sponsor_df = subset_dataframe(sponsor_df,'employer_legal_name', ['WeThinkCode_'])

**Process SDL numbers**

In [260]:
sponsor_df['sdl'] = sponsor_df['sdl'].astype(str)
sponsor_df['sdl'] = sponsor_df['sdl'].str.split(".")[0][0]
sponsor_df['sdl'] = sponsor_df['sdl'].str.strip()
sponsor_df['sdl'] = [elem[-9:] if len(elem) > 9 else elem for elem in sponsor_df['sdl']]

#use split_column
split_column_num(sponsor_df, "sdl", 9)

**Process Skills Development**

In [261]:
#create skills development binary option
column_to_binary(sponsor_df, 'skills_development')

**Process Lead Employer**

In [262]:
#create skills development binary option
column_to_binary(sponsor_df, 'lead_employer')

**Process Business Address**

In [263]:
#create business address
split_address(sponsor_df, "business_address")

**Process Business Postal Adress**

In [264]:
#create business postal address
split_address(sponsor_df, "business_postal_address")

**Split Full Name**

In [265]:
sponsor_df['learnership_contact'] = sponsor_df['employer_fullname'].str.split()[0][0]
sponsor_df['learnership_contact_name'] = sponsor_df['employer_fullname'].str.split()[0][0]
sponsor_df['learnership_contact_surname'] = sponsor_df['employer_fullname'].str.split()[0][-1]

**Verify Columns**

In [266]:
for column in list(filter(lambda x: type(x) == str,tag_ids_df['MICT'])):
    if column not in sponsor_df.columns:
        print(column)

**Set Learner Email and Template ID**

In [267]:
sponsor_df['learner_email'] = 'mufaro@thoughtquest.co.za'

template_ids = {'MICT':'31c56eea-86c8-47aa-bb88-29789c285eff', 'BANKSETA':'b464c79c-22c5-4997-b769-890e264b2635',
               'FASSET':'dbe642a5-c97b-4f9f-9980-ec785fbd38f6', 'Contract':'6c6ae58c-2d42-41e4-b9e6-73f0c2d1363d'}
sponsor_df['template_id'] = template_ids['Contract']

target_columns = list(filter(lambda x: type(x) == str,tag_ids_df['Contract']))

**Replace NULL Values**

In [268]:
sponsor_df = sponsor_df.fillna(" ")

**Send Contract**

In [269]:
send_bulk_signrequest(sponsor_df, target_columns, 'template_id')

Signer: mufaro@thoughtquest.co.za , Status: 201


In [232]:
for column in target_columns:
    print(f"{column} : {sponsor_df[column].values[0]}")

employer_initials : M.S.C
employer_legal_name : WeThinkCode_
employer_trading_name : WeThinkCode_
seta_name : MICT SETA
skills_development_yes : X
skills_development_no : 
sdl : 123456789
seta_name : MICT SETA
sic : 123456
lead_employer_yes : X
lead_employer_no : 
business_address_1 : 84 Albertina Sisulu Rd,
business_address_2 : CBD,
business_address_3 : Johannesburg,
business_postal_address_1 : nan,
business_postal_address_2 :  
business_postal_address_3 :  
learnership_contact : Mufaro
employer_phone : 835555564
employer_fax :  
employer_phone : 835555564
employer_email : mufaro@wethinkcode.co.za
employer_fullname : Mufaro Simbisayi
employer_designation : Accreditation


In [233]:
sponsor_df['employer_phone']

0    835555564
Name: employer_phone, dtype: object